# Project: Fault Monitoring and Inspection for Electric Utilities

## Description:

- The task is *`to showcase the possibility of detecting and inspecting EPTN assets of Electric Utilities in Nigeria utilizing UAV imagery and deep learning approach.`* 

- The main application of this dataset is that the continuous inspection of transmission components provides a lot of information about the state of the network. This can help us potentially monitor faults on transmission lines, monitor assets state, and optimize EPTN for Transmission companies using deep learning, and if any discrepancy is found, actions can be taken quickly to diagnose that.<p/>

# Introduction
- GPU: Tesla K80 GPU of about 12GB
- Runtime is limited to 12 hours.
- https://colab.research.google.com
- https://colab.research.google.com/notebooks/welcome.ipynb
## Tensorflow Object Detection
- https://github.com/tensorflow/models/tree/master/research/object_detection

### Mounting Google Drive to Colab

In [ ]:
#mount Google Drive Folder
from google.colab import drive
drive.mount('/content/drive')

# change to working tensorflow directory on the drive
%cd '/content/drive/My Drive/Colab/models/'

### Clone from github repo the Tensorflow models (tf version 1.15)

In [ ]:
!git clone --q https://github.com/tensorflow/models.git

# Training of Models

In [ ]:
#upgrade pip and protobuf
!pip install --upgrade pip
!pip install --upgrade protobuf

### Setting tensorflow version 1.x

In [ ]:
#install tensorflow version 1.15 and numpy
%tensorflow_version 1.15
import tensorflow as tf
print(tf.__version__)

!pip install numpy

In [ ]:
#check your GPU status
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

### Import libraries, Create a Python path, and build and install Tensorflow object detection model 

In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
%cd /content/drive/My Drive/Colab/models/research/
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Colab/models/research/:/content/drive/My Drive/Colab/models/research/slim'

!python setup.py build
!python setup.py install

In [ ]:
#!python object_detection/builders/model_builder_test.py

In [ ]:
#performing the training of models
!pip install tf_slim
%cd /content/drive/My Drive/models/research/Object_Detection
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Colab/models/research/:/content/drive/My Drive/models/research/slim'

### Time Configuration

In [ ]:
import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

### Config models to use

In [ ]:
MODELS_CONFIG = {
    'ssd_mobnet_v1': {
        'model_name': 'ssd_mobilenet_v1_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync.config',
    },
    'ssd_rest50_v1': {
        'model_name': 'ssd_resnet50_v1_coco_2018_01_28',
        'pipeline_file': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config',
    }
    'ssd_rest101_v1': {
        'model_name': 'ssd_resnet101_v1_oid_2018_01_28',
        'pipeline_file': 'ssd_resnet101_v1_fpn_shared_box_predictor_oid_640x640_sync.config',
    }
}

selected_model = 'ssd_mobnet_v1'

### Download pretrained model

In [ ]:
%cd /content/drive/My Drive/models/research

MODEL = MODELS_CONFIG[selected_model]['model_name']

pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

fine_tune_dir = '/content/drive/My Drive/Colab/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

### Setting Tensorboard

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir 'training'

### Train the model

In [ ]:
#ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config
#ssd_resnet101_v1_fpn_shared_box_predictor_oid_640x640_sync.config
#ssd_mobilenet_v1_fpn_shared_box_predictor_640x640_coco14_sync.config
!python legacy/train.py --train_dir=training/ --pipeline_config_path=training/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config --logtostderr

OR

In [ ]:
!python /content/drive/My Drive/Colab/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_path}\
    --model_dir={model_dir} \
    --alsologtostderr \

### Export Inference Graph

In [ ]:
#the location where the exported model will be saved in.
output_directory = '/content/drive/My Drive/Colab/models/research/fine_tuned_model'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

!python /content/drive/My Drive/Colab/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_path} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

<p align = "center"> OR </p>

In [ ]:
#the XXXX is dependent on the number of steps in the config files from the pretrained model
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config --trained_checkpoint_prefix training/model.ckpt-XXXX --output_directory ssdrest50_new_graph

### Downloads the frozen model for inference

In [ ]:
from google.colab import files
files.download(output_directory + '/frozen_inference_graph.pb')

In [ ]:
#converting the model graph to a zip folder before downloading
!zip -r ssdrest50_model_graph.zip ssdrest50_new_graph

In [ ]:
import numpy as np
import os
%tensorflow_version 1.x
import tensorflow as tf
import cv2
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from google.colab.patches import cv2_imshow

# path to the frozen graph:
PATH_TO_FROZEN_GRAPH = '/content/drive/My Drive/Colab/models/research/fine_tuned_model/frozen_inference_graph.pb'

# path to the label map
PATH_TO_LABEL_MAP = 'content/drive/My Drive/Colab/models/data/labelmap.pbtxt'

PATH_TO_IMAGE = "/content/gdrive/My Drive/Colab/models/images/test/000028.jpg"

# number of classes 
NUM_CLASSES = 2


label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)


image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_expanded = np.expand_dims(image, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visulaize the results')

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.80)

# All the results have been drawn on image. Now display the image.
cv2_imshow(image)
filename = "/content/gdrive/My Drive/Colab/models/images/tested_ssd_mobilenet_v2_coco/tl_tested_inference_5.jpg"
cv2.imwrite(filename, image)

# Press any key to close the image
cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

IMAGE_SIZE = (12, 8)

plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image)